# **Нарезка изображений под YOLOv8 для до-обучения**

# **Сервисные функции**

In [1]:
import time

# @title Контекстный менеджер для измерения времени операций
class timex:
    def __enter__(self):
        # Фиксация времени старта процесса
        self.t = time.time()
        return self

    def __exit__(self, type, value, traceback):
        # Вывод времени работы
        # Расчет времени выполнения
        result_time = time.time()-self.t
        hour = int(result_time//3600)
        min = int(result_time//60)-hour*60
        sec = int(round(result_time%60))
        msec = round(1000*result_time%60)

        if hour > 0:
          print('\nВремя обработки: ' + str(hour)+' час. ' + str(min)+' мин.')
        elif min > 0:
          print('\nВремя обработки: ' + str(min)+' мин. ' + str(sec)+' сек.')
        elif sec > 0:
          print('\nВремя обработки: ' + str(sec)+' сек.')
        else:
          print('\nВремя обработки: ' + str(msec)+' мс.')


In [2]:
import gdown

# @title Функция загрузки видео в Google Colab
def load_video(video_url, video_name):
    # Стартовая отметка времени
    t_0 = time.time()

    # Загружаем видео в Colab
    gdown.download(video_url, None, quiet=True)

    video_path = f"/content/{video_name}"

    # Расчет времени выполнения
    t_f = time.time()-t_0

    # Расчет времени обработи и приведение его к легко читаемому виду
    time_text = str(int(round(t_f//60,0)))+' мин '+str(int(round(t_f%60,0)))+' сек.'

    # Подведение итогов обработки
    print('Загрузка видеофайла: '+ video_name + '\nсделана за: ' + time_text)

    return video_path


In [3]:
import cv2
from datetime import timedelta

# @title Функция для создания списка временных меток (time_list)
def generate_time_list(video_path, interval_seconds, start_time=None, end_time=None):
    # Открываем видеофайл
    cap = cv2.VideoCapture(video_path)

    # Проверяем, успешно ли открыт файл
    if not cap.isOpened():
        print(f"Не удалось открыть видеофайл: {video_path}")
        return None

    # Получаем общее количество кадров и частоту кадров
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Рассчитываем интервал в кадрах
    interval_frames = int(interval_seconds * fps)

    # Рассчитываем начальный и конечный кадры
    start_frame = 0 if start_time is None else int((start_time[0] * 60 + start_time[1]) * fps)
    end_frame = total_frames if end_time is None else int((end_time[0] * 60 + end_time[1]) * fps)

    # Генерируем временные метки
    time_list = []

    for frame_time in range(start_frame, end_frame, interval_frames):
        # Преобразуем время в минуты и секунды
        time_in_seconds = frame_time / fps
        time_minutes = int(time_in_seconds // 60)
        time_seconds = int(time_in_seconds % 60)

        # Добавляем временную метку в список
        time_list.append((time_minutes, time_seconds))

    # Отключаем видеопоток
    cap.release()

    return time_list


In [4]:
from google.colab.patches import cv2_imshow
import cv2
from pathlib import Path

# @title Функция извлечения кадров из видео по указанным временным меткам
def extract_frames_from_video(video_path, time_list, path_output):
    # Стартовая отметка времени
    t_0 = time.time()

    # Открываем видеофайл
    cap = cv2.VideoCapture(video_path)

    # Проверяем, успешно ли открыт файл
    if not cap.isOpened():
        print(f"Не удалось открыть видеофайл: {video_path}")
        return None

    frames = []  # Список для хранения извлеченных кадров

    # Обрабатываем каждую временную метку
    for time_point in time_list:
        # Рассчитываем время в кадрах
        target_frame_time = int((time_point[0] * 60 + time_point[1]) * cap.get(cv2.CAP_PROP_FPS))

        # Устанавливаем позицию в видео на нужный кадр
        cap.set(cv2.CAP_PROP_POS_FRAMES, target_frame_time)

        # Считываем кадр
        ret, frame = cap.read()

        # Проверяем, успешно ли считан кадр
        if not ret:
            print(f"Не удалось считать кадр для времени {time_point[0]} минут {time_point[1]} секунд.")
            continue

        # Добавляем кадр в список
        frames.append(frame)

    # Отключаем видеопоток
    cap.release()

    # Расчет времени выполнения
    t_f = time.time() - t_0

    # Расчет времени обработи и приведение его к легко читаемому виду
    time_text = str(int(round(t_f // 60, 0))) + ' мин ' + str(int(round(t_f % 60, 0))) + ' сек.'

    # Подведение итогов обработки
    print(f'Из видеофайла: {video_path}\nизвлечены кадры для времен: {time_list}\nза: {time_text}\n')

    # Создаем папку Image, если она не существует
    Path(path_output).mkdir(parents=True, exist_ok=True)

    folder_name = os.path.basename(os.path.normpath(path_output)).lower()

    # Сохраняем извлеченные кадры в папке Image
    for i, frame in enumerate(frames):
        # Формируем имя файла на основе времени и сохраняем в папке Image
        file_name = f"{folder_name}_{time_list[i][0]:02d}m_{time_list[i][1]:02d}s.png"
        file_path = Path(path_output) / file_name
        # cv2.imwrite(str(file_path), cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        cv2.imwrite(str(file_path), frame)  # Убрали cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Выводим информацию о сохраненном файле
        print(f"Сохранен файл: {file_path}")
        print(f"Разрешение файла: {frame.shape[1]}x{frame.shape[0]} пикселей\n")

    return frames


In [5]:
# @title Функция изменения ширины кадра (высота пропорционально)
def show_resized_images(frames, new_width):
    """
    Отображает изображения с установленной новой шириной, сохраняя пропорции.

    Параметры:
    - frames: List[numpy.ndarray], список входных изображений.
    - new_width: int, новая ширина изображения.
    """
    for i, frame in enumerate(frames):
        aspect_ratio = frame.shape[1] / frame.shape[0]
        new_height = int(new_width / aspect_ratio)
        resized_image = cv2.resize(frame, (new_width, new_height))
        cv2_imshow(resized_image)
        print()


In [6]:
import shutil
import zipfile

# @title Функция архивации содержимого папки и сохранения архива по указанному пути
def zip_folder(folder_path, zip_path):
    """
    Архивирует содержимое папки и сохраняет архив по указанному пути.

    Параметры:
    - folder_path: str, путь к архивируемой папке.
    - zip_path: str, путь, по которому сохраняется zip-архив.
    """
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arc_name = os.path.relpath(file_path, folder_path)
                zipf.write(file_path, arcname=arc_name)


In [7]:
import os
import cv2
import matplotlib.pyplot as plt

# @title Функция нарезки изображения на фрагменты по размеру со смещением-наложением
def crop_and_save_images_by_fragment_size_and_shift(input_folder,
                                                    output_folder,
                                                    fragment_size=640,
                                                    shift_pixels=320,
                                                    quiet=True):
    """
    Нарезает изображения из указанной папки на фрагменты размером fragment_size x fragment_size с перекрытием shift_pixels
    и сохраняет их.

    Параметры:
    - input_folder: str, путь к папке с изображениями.
    - output_folder: str, путь к папке для сохранения результата.
    - fragment_size: int, размер фрагмента.
    - shift_pixels: int, смещение по горизонтали и вертикали при нарезке фрагментов изображения.
    - quiet: bool, флаг для подавления вывода изображений на экран (по умолчанию True).
    """

    # folder_name = os.path.basename(os.path.normpath(input_folder))

    # Создаем каталог для сохранения результата
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

    # Получаем список файлов и сортируем его по имени
    file_list = sorted(os.listdir(input_folder))

    # Проходим по всем файлам в папке
    for idx, file_name in enumerate(file_list):
        # Проверяем, что файл является изображением
        if file_name.endswith(('.png', '.jpg', '.jpeg')):
            # Полный путь к файлу
            file_path = os.path.join(input_folder, file_name)

            # Загрузка изображения
            img = cv2.imread(file_path, cv2.IMREAD_COLOR)

            # Извлечение имени файла без расширения
            base_name, _ = os.path.splitext(file_name)

            # Создаем каталог для сохранения фрагментов
            cropped_subfolder = os.path.join(output_folder, base_name + '_cropped')
            os.makedirs(cropped_subfolder, exist_ok=True)

            # Сохраняем оригинальное изображение в папке с нарезанными фрагментами
            original_file_path = os.path.join(cropped_subfolder, f"{base_name}_original.png")
            cv2.imwrite(original_file_path, img)

            # Выводим оригинал на печать только для первого и последнего файла
            if quiet and (idx == 0 or idx == len(file_list) - 1):
                plt.figure(figsize=(20, 12))
                plt.imshow(img[:, :, ::-1])
                plt.title(f"Original - {base_name}")
                plt.axis('off')
                plt.show()

            # Размеры изображения после обрезки
            height, width, _ = img.shape

            # Вычисляем количество фрагментов по ширине и высоте
            num_width_fragments = (width - fragment_size) // shift_pixels + 1
            num_height_fragments = (height - fragment_size) // shift_pixels + 1

            # Выводим изображения для проверки только для первого и последнего файла
            if quiet and (idx == 0 or idx == len(file_list) - 1):
                plt.figure(figsize=(20, 15))

            for i in range(num_height_fragments):
                for j in range(num_width_fragments):
                    start_h = i * shift_pixels
                    start_w = j * shift_pixels

                    # Обрезаем фрагмент с учетом смещения
                    sub_img = img[start_h:start_h + fragment_size, start_w:start_w + fragment_size, ::-1]

                    sub_name = f"sz{fragment_size}_#{i * num_width_fragments + j + 1}"

                    if quiet and (idx == 0 or idx == len(file_list) - 1):
                        # Размещаем фрагмент изображения на соответствующем подграфике
                        ax = plt.subplot(num_height_fragments + 1, num_width_fragments + 1, i * num_width_fragments + j + 1)
                        ax.get_xaxis().set_visible(False)
                        ax.get_yaxis().set_visible(False)
                        ax.set_title(sub_name)
                        plt.imshow(sub_img)

                    # Сохраняем фрагмент в папку
                    sub_file_name = f"{base_name}_{sub_name}.png"
                    sub_file_path = os.path.join(cropped_subfolder, sub_file_name)
                    cv2.imwrite(sub_file_path, sub_img[:, :, ::-1])

            # Обработка остатка по высоте
            if height % shift_pixels != 0:
                start_h = height - fragment_size

                # Пройти по ширине с шагом fragment_size
                for k in range(num_width_fragments):
                    start_w = k * shift_pixels
                    sub_img = img[start_h:start_h + fragment_size, start_w:start_w + fragment_size, ::-1]

                    sub_name = f"sz{fragment_size}_#{num_height_fragments * num_width_fragments + k + 1}"

                    #print(f"Creating subplot {i + 1} with name {sub_name}")

                    if quiet and (idx == 0 or idx == len(file_list) - 1):
                        # Размещаем фрагмент изображения на соответствующем подграфике
                        ax = plt.subplot(num_height_fragments + 1, num_width_fragments + 1, num_height_fragments * num_width_fragments + k + 1)
                        ax.get_xaxis().set_visible(False)
                        ax.get_yaxis().set_visible(False)
                        ax.set_title(sub_name)
                        plt.imshow(sub_img)

                    # Сохраняем фрагмент в папку
                    sub_file_name = f"{base_name}_{sub_name}.png"
                    sub_file_path = os.path.join(cropped_subfolder, sub_file_name)
                    cv2.imwrite(sub_file_path, sub_img[:, :, ::-1])

            # Обработка остатка по ширине
            if width % shift_pixels != 0:
                start_w = width - fragment_size

                # Пройти по высоте с шагом fragment_size
                for k in range(num_height_fragments):
                    start_h = k * shift_pixels
                    sub_img = img[start_h:start_h + fragment_size, start_w:start_w + fragment_size, ::-1]

                    sub_name = f"sz{fragment_size}_#{(num_height_fragments + 1) * num_width_fragments + k + 1}"

                    if quiet and (idx == 0 or idx == len(file_list) - 1):
                        # Размещаем фрагмент изображения на соответствующем подграфике
                        ax = plt.subplot(num_height_fragments + 1, num_width_fragments + 1, (num_height_fragments + 1) * num_width_fragments + k + 1)
                        ax.get_xaxis().set_visible(False)
                        ax.get_yaxis().set_visible(False)
                        ax.set_title(sub_name)
                        plt.imshow(sub_img)

                    # Сохраняем фрагмент в папку
                    sub_file_name = f"{base_name}_{sub_name}.png"
                    sub_file_path = os.path.join(cropped_subfolder, sub_file_name)
                    cv2.imwrite(sub_file_path, sub_img[:, :, ::-1])

            # Обработка остатка по высоте и по ширине (правый нижний угол изображения)
            if height % shift_pixels != 0 and width % shift_pixels != 0:
                start_h = height - fragment_size
                start_w = width - fragment_size

                sub_img = img[start_h:start_h + fragment_size, start_w:start_w + fragment_size, ::-1]

                sub_name = f"sz{fragment_size}_#{(num_height_fragments + 1) * (num_width_fragments + 1)}"

                if quiet and (idx == 0 or idx == len(file_list) - 1):
                    # Размещаем фрагмент изображения на соответствующем подграфике
                    ax = plt.subplot(num_height_fragments + 1, num_width_fragments + 1, (num_height_fragments + 1) * (num_width_fragments + 1))
                    ax.get_xaxis().set_visible(False)
                    ax.get_yaxis().set_visible(False)
                    ax.set_title(sub_name)
                    plt.imshow(sub_img)

                # Сохраняем фрагмент в папку
                sub_file_name = f"{base_name}_{sub_name}.png"
                sub_file_path = os.path.join(cropped_subfolder, sub_file_name)
                cv2.imwrite(sub_file_path, sub_img[:, :, ::-1])

            if quiet and (idx == 0 or idx == len(file_list) - 1):
                # Удаление осей для избежания предупреждения
                ax.remove()


In [8]:
import gdown
import zipfile
import os
import matplotlib.pyplot as plt
from PIL import Image

# @title Функция загрузки zip с изображениями и отображение их на полотне
def download_and_display_images(url, num_rows=3, num_cols=2, figsize=(10, 10)):
    """
    Загружает изображения из zip-файла по указанной ссылке и отображает их на полотне.

    Параметры:
    - url: str, ссылка на zip-файл с изображениями.
    - num_rows: int, количество строк на полотне.
    - num_cols: int, количество столбцов на полотне.
    - figsize: tuple, размер полотна (ширина, высота).

    Пример использования:
    download_and_display_images('ссылка_на_ваш_файл.zip', num_rows=3, num_cols=2, figsize=(10, 10))
    """
    # Загружаем файл zip
    output_zip = 'downloaded_images.zip'
    gdown.download(url, output_zip, quiet=False)

    # Удаляем все файлы в output_folder
    output_folder = 'extracted_images'
    if os.path.exists(output_folder):
        for file_name in os.listdir(output_folder):
            file_path = os.path.join(output_folder, file_name)
            os.remove(file_path)
    else:
        os.makedirs(output_folder)

    # Разархивируем файл
    output_folder = 'extracted_images'
    with zipfile.ZipFile(output_zip, 'r') as zip_ref:
        zip_ref.extractall(output_folder)

    # Получаем список файлов изображений в папке, отсортированный по именам
    image_files = sorted([file for file in os.listdir(output_folder) if file.endswith(('.png', '.jpg', '.jpeg'))])

    # Создаем полотно для изображений
    plt.figure(figsize=figsize)

    # Отображаем изображения на полотне
    for i, image_file in enumerate(image_files, start=1):
        img_path = os.path.join(output_folder, image_file)
        img = Image.open(img_path)
        plt.subplot(num_rows, num_cols, i)
        plt.imshow(img)
        # plt.title(f"Image {i}")
        plt.title(f"{image_file}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()


In [9]:
from google.colab import drive
import shutil
import os

# @title Функция копирования zip-архивов на подключённый Google Disk в папку destination_folder
def copy_archives_to_destination(destination_folder):
    # Пути к архивам
    zip_path_cropped_1 = '/content/CroppedVid1.zip'
    zip_path_cropped_2 = '/content/CroppedVid2.zip'
    zip_path_1 = '/content/Vid1.zip'
    zip_path_2 = '/content/Vid2.zip'

    # Создание всех промежуточных папок, если они не существуют
    os.makedirs(destination_folder, exist_ok=True)

    # Копирование архивов
    shutil.copy(zip_path_cropped_1, destination_folder)
    shutil.copy(zip_path_cropped_2, destination_folder)
    shutil.copy(zip_path_1, destination_folder)
    shutil.copy(zip_path_2, destination_folder)


# **Загрузка видео в Google Colab**

In [10]:
# @title Загрузка видео-файла .MP4 из папки 178.31.03.2023...
video_url = 'https://drive.google.com/uc?id=1T3-v-B-fgWv3gVirog2q3DAlw34JK5gd'  # Общий аккаунт

video_name = 'DJI_0001 (9).MP4'

# Загрузка видео
video_path_1 = load_video(video_url, video_name)


Загрузка видеофайла: DJI_0001 (9).MP4
сделана за: 0 мин 42 сек.


In [11]:
# @title Загрузка видео-файла .MP4 из папки 173.24.02.2023...
video_url = 'https://drive.google.com/uc?id=1zLaPIR-QNzpuhGr_DL9SSmI8vRcrwSj6'  # Общий аккаунт

video_name = 'DJI_0001 (1).MP4'

# Загрузка видео
video_path_2 = load_video(video_url, video_name)


Загрузка видеофайла: DJI_0001 (1).MP4
сделана за: 0 мин 27 сек.


# **Извлечение кадров из видео-файла**

In [12]:
# @title Извлечение кадров для строительной техники из видео в папке 178.31.03.2023

# time_list_1 = [(0, 30), (1, 0), (2, 15)]  # Список временных меток в формате (минуты, секунды)
path_output_1 = '/content/Vid1'

# Создание списка временных меток (time_list)
interval_seconds_1 = 10  # интервал в секундах
start_time_1 = (3, 49)
end_time_1 = (4, 54)
time_list_1 = generate_time_list(video_path_1, interval_seconds=interval_seconds_1, start_time=start_time_1, end_time=end_time_1)


frames_1 = extract_frames_from_video(video_path_1, time_list_1, path_output_1)

# Вывод изображения
if frames_1 and len(frames_1) >= 2:
    show_resized_images([frames_1[0], frames_1[-1]], new_width=1500)
elif frames_1:
    show_resized_images([frames_1[0]], new_width=1500)


Output hidden; open in https://colab.research.google.com to view.

In [13]:
# @title Извлечение кадров для строительной техники из видео в папке 173.24.02.2023

# time_list_2 = [(0, 30), (1, 0), (2, 15)]  # Список временных меток в формате (минуты, секунды)
path_output_2 = '/content/Vid2'

# Создание списка временных меток (time_list)
interval_seconds_2 = 10  # интервал в секундах
start_time_2 = (3, 13)
end_time_2 = (4, 14)
time_list_2 = generate_time_list(video_path_2, interval_seconds=interval_seconds_2, start_time=start_time_2, end_time=end_time_2)

frames_2 = extract_frames_from_video(video_path_2, time_list_2, path_output_2)

# Вывод изображения
if frames_2 and len(frames_2) >= 2:
    show_resized_images([frames_2[0], frames_2[-1]], new_width=1500)
elif frames_2:
    show_resized_images([frames_2[0]], new_width=1500)


Output hidden; open in https://colab.research.google.com to view.

In [14]:
# @title Архивация содержимого папок и сохранение архивов по указанному пути
folder_to_zip = '/content/Vid1'
zip_output_path = '/content/Vid1.zip'

zip_folder(folder_to_zip, zip_output_path)

folder_to_zip = '/content/Vid2'
zip_output_path = '/content/Vid2.zip'

zip_folder(folder_to_zip, zip_output_path)


In [15]:
# @title Нарезка изображения из Vid1 на фрагменты 640x640 со смещением 576 пикселей (90%)
input_folder = '/content/Vid1/'
output_folder = '/content/CroppedVid1/'

crop_and_save_images_by_fragment_size_and_shift(input_folder, output_folder, fragment_size=640, shift_pixels=576, quiet=True)


Output hidden; open in https://colab.research.google.com to view.

In [16]:
# @title Нарезка изображения из Vid1 на фрагменты 1280x1280 со смещением 1152 пикселей (90%)
input_folder = '/content/Vid1/'
output_folder = '/content/CroppedVid1/'

crop_and_save_images_by_fragment_size_and_shift(input_folder, output_folder, fragment_size=1280, shift_pixels=1152, quiet=True)


Output hidden; open in https://colab.research.google.com to view.

In [17]:
# @title Архивация содержимого папки Vid1 с нарезками и сохранение архива
folder_to_zip = '/content/CroppedVid1'
zip_output_path = '/content/CroppedVid1.zip'

zip_folder(folder_to_zip, zip_output_path)


In [18]:
# @title Нарезка изображения из Vid2 на фрагменты 640x640 со смещением 576 пикселей (90%)
input_folder = '/content/Vid2/'
output_folder = '/content/CroppedVid2/'

crop_and_save_images_by_fragment_size_and_shift(input_folder, output_folder, fragment_size=640, shift_pixels=576, quiet=True)


Output hidden; open in https://colab.research.google.com to view.

In [19]:
# @title Нарезка изображения из указанной папки на фрагменты 1280x1280 со смещением 1152 пикселей (90%)
input_folder = '/content/Vid2/'
output_folder = '/content/CroppedVid2/'

crop_and_save_images_by_fragment_size_and_shift(input_folder, output_folder, fragment_size=1280, shift_pixels=1152, quiet=True)


Output hidden; open in https://colab.research.google.com to view.

In [20]:
# @title Архивация содержимого папки с нарезками и сохранение архива
folder_to_zip = '/content/CroppedVid2'
zip_output_path = '/content/CroppedVid2.zip'

zip_folder(folder_to_zip, zip_output_path)


In [21]:
# Подключение к Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# @title Копирование zip-архивов на подключённый Google Disk в папку destination_folder
custom_destination_folder = '/content/drive/MyDrive/Intership_03/Bases/Zip_equip_2video'

# Вызов функции с пользовательским путем для выполнения копирования
copy_archives_to_destination(custom_destination_folder)


# **Описание к архивам изображений**

**Итого обработано 2 видео**

Кадры из видео извлекались с интервалом 10 сек. Разрешение кадров 3840х2160.

Из 1-го видео извлечено 7 кадров с 3:49 по 4:49.

Из 2-го видео извлечено 7 кадров с 3:13 по 4:13.

**Итого 14 кадров 3840х2160.**

Далее эти кадры были нарезаны фрагментами 640х640 с наложением-смещением 576 пикселей (90%), 1280х1280 с наложением-смещением 1152 пикселей (90%). Наложение-смещение производилось как по горизонтали так и по вертикали.

Ширина делится кратно на смещения и получается целое число итераций.

В данном случае и по ширине и по высоте **НЕ** делится кратно, т.е. получаются остатки, которые тоже используются. Т.е. в конце двойного цикла по горизонтали и вертикали ещё раз проходим по ширине с учётом остатка, затем ещё раз проходим по высоте с учётом остатка и в конце не забываем про нижний правый угол изображения.

Итого на каждый оригинальный кадр 3840х2160 получается:

15 фрагментов с разрешением 640х640

6 фрагментов с разрешением 1280х1280

1 оригинальный кадр с разрешением 3840х2160

Таким образом, **из 1-го кадра получаем 22 изображения.**

---

**Структура архивов:**

**Vid1.zip:** Просто оригинальные кадры с разрешением 3840х2160 из 1-го видео.

**CroppedVid1.zip:** Каждый кадр из 1-го видео имеет свою папку с именем типа "vid1_02m_19s_cropped", где указано, что это видео 1, кадр извлечён с временной метки 2 мин 19 сек.

Внутри этой папки находятся сами фрагменты с именем типа "vid1_02m_19s_sz640_#61.png", "vid1_02m_19s_sz1280_#11.png", "vid1_02m_19s_original.png". По которому можно понять какого размера фрагмент и какой его порядковый номер. Либо это оригинальный кадр.

Аналогично с архивами из 2-го видео.